In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [ ]:
train_data=pd.read_csv('../input/digit-recognizer/train.csv',dtype=np.float32)

In [ ]:
train_data.head()

In [ ]:
X=train_data.drop('label',axis=1).values
y=train_data['label'].values

In [ ]:
X=X/255

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def convert_features(name):
    name=torch.from_numpy(name)
    return name
def convert_target(name):
    name=torch.from_numpy(name).type(torch.LongTensor)
    return name

In [ ]:
X_train=convert_features(X_train)
X_test=convert_features(X_test)
y_train=convert_target(y_train)
y_test=convert_target(y_test)

In [ ]:
train=torch.utils.data.TensorDataset(X_train,y_train)
test=torch.utils.data.TensorDataset(X_test,y_test)

In [ ]:
train_loader=DataLoader(dataset=train,batch_size=100,shuffle=False)
test_loader=DataLoader(dataset=test,batch_size=100,shuffle=False)

In [ ]:
plt.imshow(X_train[10].reshape(28,28))
plt.axis("off")
plt.title(str(y_train[10]))
plt.savefig('graph.png')
plt.show()

In [ ]:
class NET(nn.Module):
    def __init__(self,input_shape,output_shape):
        super(NET,self).__init__()
        self.f1=nn.Linear(input_shape,output_shape)
    def forward(self,x):
        x=self.f1(x)
        return x

In [ ]:
net=NET(28*28,10)

In [ ]:
net

In [ ]:
# Cross Entropy Loss  
error = nn.CrossEntropyLoss()

# SGD Optimizer 
learning_rate = 0.001
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [ ]:
batch_size=100
n_iters = 10000
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)

In [ ]:
# Traning the Model
count = 0
loss_list = []
iteration_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        # Define variables
        train = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = net(train)
        # Calculate softmax and cross entropy loss
        loss = error(outputs, labels)
        
        # Calculate gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        # Prediction
        if count % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Predict test dataset
            for images, labels in test_loader: 
                test = Variable(images.view(-1, 28*28))
                
                # Forward propagation
                outputs = net(test)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += len(labels)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}  Accuracy: {}%'.format(count, loss.data, accuracy))

In [ ]:
plt.plot(iteration_list,loss_list)

In [ ]:
test_data=pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
test_data.head()

In [ ]:
X=test_data.values/255

In [ ]:
X=torch.from_numpy(X).type(torch.float32)

In [ ]:
Xtest_data=torch.utils.data.TensorDataset(X)

In [ ]:
test_data=DataLoader(dataset=test_data,batch_size=100,shuffle=False)

In [ ]:
test_data

In [ ]:
pred=[]
for images in test_data:
    tt=Variable(images.view(-1,28,28))
    outputs=net(tt)
    predictions=torch.max(outputs.data,1)[1]
    pred.append(predictions)

In [ ]:
test=Variable(X[0].view(-1,784))

In [ ]:
output=net(test)

In [ ]:
predicted=torch.max(output.data,1)[1]

In [ ]:
predicted

In [ ]:
len(X)

In [ ]:
# Making a loop function for the same
predictions=[]
for i in X:
    test=Variable(i.view(-1,784))
    output=net(test)
    predicted=torch.max(output.data,1)[1]
    predictions.append(predicted)

In [ ]:
sample=pd.DataFrame()

In [ ]:
idd=[i for i in range(1,28001)]

In [ ]:
len(predictions)

In [ ]:
len(idd)

In [ ]:
sample['ImageId']=idd
sample['Label']=predictions

In [ ]:
for i in range(len(predictions)):
    predictions[i]=int(predictions[i][0])

In [ ]:
sample.head()

In [ ]:
sample.to_csv('SAmple_predictions.csv')